In [116]:
import pandas as pd
import psycopg2

In [117]:
file_path = r'\\10.98.16.69\dle\Materiales'
excel_name = 'EXPORT_20240806_213729.xlsx'

file_path = file_path + '\\' + excel_name

In [118]:
df = pd.read_excel(file_path, sheet_name = 'Data')

In [119]:
print(df.head())

    Plant      Material               Material Description  Material Document  \
0  2311.0  230000011124  SCREW PT DELTA - 30 x 1.12 x 10MM       4.051103e+09   
1  2311.0  230000051021           LEXAN 141R-703 GREY (PC)       4.051103e+09   
2  2311.0  230000051021           LEXAN 141R-703 GREY (PC)       4.050876e+09   
3  2311.0  230000051021           LEXAN 141R-703 GREY (PC)       4.049913e+09   
4  2311.0  230000051029          MM ACRY V-825-461 Med Red       4.051103e+09   

   Material Doc.Item  Movement Type  Qty in unit of entry Unit of Entry  \
0                2.0          552.0               5000.00            PC   
1                9.0          551.0                -19.03            KG   
2                2.0          552.0                399.91            KG   
3                1.0          551.0                -50.00            KG   
4                6.0          551.0                -32.83            KG   

   Amt.in Loc.Cur. Currency Posting Date User Name Movement Ty

In [120]:
# Lista de columnas a mantener
columnas_a_mantener = ['Material', 'Material Description', 'Material Document', 'Movement Type', 'Qty in unit of entry', 'Unit of Entry', 'Local Entry Time']

# Mantener solo las columnas especificadas en la lista
df_cleaned = df[columnas_a_mantener]

print(df_cleaned.head())

       Material               Material Description  Material Document  \
0  230000011124  SCREW PT DELTA - 30 x 1.12 x 10MM       4.051103e+09   
1  230000051021           LEXAN 141R-703 GREY (PC)       4.051103e+09   
2  230000051021           LEXAN 141R-703 GREY (PC)       4.050876e+09   
3  230000051021           LEXAN 141R-703 GREY (PC)       4.049913e+09   
4  230000051029          MM ACRY V-825-461 Med Red       4.051103e+09   

   Movement Type  Qty in unit of entry Unit of Entry      Local Entry Time  
0          552.0               5000.00            PC  08/04/2024  04:33:35  
1          551.0                -19.03            KG  08/04/2024  05:35:08  
2          552.0                399.91            KG  08/02/2024  16:15:35  
3          551.0                -50.00            KG  07/29/2024  23:10:47  
4          551.0                -32.83            KG  08/04/2024  05:35:08  


In [121]:
# Eliminar filas donde todas las columnas son NaN
df_cleaned = df_cleaned.dropna(how='all')

# Eliminar filas con valores nulos en columnas críticas
df_cleaned = df_cleaned.dropna(subset=['Material', 'Movement Type', 'Qty in unit of entry'])

# Convertir tipos de datos si es necesario
df_cleaned['Material'] = df_cleaned['Material'].astype(str)
df_cleaned['Material Description'] = df_cleaned['Material Description'].astype(str)
df_cleaned['Material Document'] = df_cleaned['Material Document'].astype(str).str.split('.').str[0]
df_cleaned['Movement Type'] = df_cleaned['Movement Type'].astype(str).str.split('.').str[0]
df_cleaned['Qty in unit of entry'] = df_cleaned['Qty in unit of entry'].astype(object)
df_cleaned['Unit of Entry'] = df_cleaned['Unit of Entry'].astype(str)
df_cleaned['Local Entry Time'] = pd.to_datetime(df_cleaned['Local Entry Time'])

print(df_cleaned.head(200))
# Calcular el número de filas eliminadas
filas_eliminadas = len(df) - len(df_cleaned)
print(f"Se eliminaron {filas_eliminadas} filas.")


         Material                 Material Description Material Document  \
0    230000011124    SCREW PT DELTA - 30 x 1.12 x 10MM        4051102997   
1    230000051021             LEXAN 141R-703 GREY (PC)        4051103061   
2    230000051021             LEXAN 141R-703 GREY (PC)        4050876028   
3    230000051021             LEXAN 141R-703 GREY (PC)        4049913467   
4    230000051029            MM ACRY V-825-461 Med Red        4051103061   
..            ...                                  ...               ...   
195  230002249010  HOUSING SIDE REPEATERGOLF VW 370 LH        4050361334   
196  230002249010  HOUSING SIDE REPEATERGOLF VW 370 LH        4050361260   
197  230002249010  HOUSING SIDE REPEATERGOLF VW 370 LH        4050361332   
198  230002249010  HOUSING SIDE REPEATERGOLF VW 370 LH        4050361267   
199  230002249010  HOUSING SIDE REPEATERGOLF VW 370 LH        4050361268   

    Movement Type Qty in unit of entry Unit of Entry    Local Entry Time  
0           

# Connect to the database

In [122]:
import config
conn = psycopg2.connect(
    host = config.HOST,
    database = config.DATABASE,
    user = config.USER,
    password = config.PASSWORD
)


In [123]:
cur = conn.cursor()

for row in df_cleaned.itertuples(index=False):
    cur.execute("INSERT INTO sch_pbi_bronze.tbl_materials_bronze(material, material_description, material_document, movement_type, qty_in_unit_of_entry, unit_of_entry, local_entry_time) VALUES (%s, %s, %s, %s, %s, %s, %s)", row)

conn.commit()

In [124]:
cur.close()
conn.close()